In [37]:
!sudo apt-get install libmagickwand-dev
!pip install --no-cache-dir \
    opencv-python-headless==4.6.*\
    rawpy==0.17.* \
    pandas \
    Pillow==7.1.2 \
    scikit-image==0.16.2 \
    scipy \
    tqdm \
    Wand

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmagickwand-dev is already the newest version (8:6.9.7.4+dfsg-16ubuntu6.14).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
!git clone https://github.com/andreacos/BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation

fatal: destination path 'BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation' already exists and is not an empty directory.


In [39]:
import sys

sys.path.insert(1, "BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation")

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [40]:
import io
import os
from glob import glob

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from networks import (custom_mse_wrapper, custom_softmax_activation,
                      custom_two_terms_loss_wrapper)
from PIL import Image
from skimage.util import view_as_windows
from utils import label2coefficient, max_min_coefficient, string2Q

In [41]:
model_file = 'BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation/models/model_QF1_60-98_QF2_90-2-term-loss.h5'
image_file =  'BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation/resources/00000000_redaf7d93t.TIF_85_90.png'
_map = np.load('BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation/resources/qf1_qf2_map_90.npy', allow_pickle=True)

In [42]:
def preprocess(image: np.array, dimension: tuple) -> np.array:
    if isinstance(image, str):
        image = cv2.imread(image)    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)[:,:,0]
    if image.shape != dimension:
        image = cv2.resize(image, dimension)
    return image.astype(np.float32) / 255.

#WMSE
def mse_weighted(pred: np.array, dct_coeffs: np.array, weights: np.array):
  diff = np.asarray((pred - dct_coeffs)**2) * weights
  weighted = np.average(diff, axis=1, weights=weights)
  return weighted

#MSE
def mse(pred: np.array, dct_coeffs: np.array):
    return np.mean((pred - dct_coeffs) ** 2, axis=1)


def make_predictWMSE(image: np.array, arr: np.array, maximum_coefficients: np.array, q50_coeffs: np.array) -> np.array:
    predicted_label = label2coefficient(model.predict(np.expand_dims(image, [0, -1]), verbose=0).flatten(), maximum_coefficients)
    return labels[np.argmin(mse_weighted(predicted_label, arr, q50_coeffs))]

def make_predictMSE(image: np.array, arr: np.array, maximum_coefficients: np.array) -> np.array:
    predicted_label = label2coefficient(model.predict(np.expand_dims(image, [0, -1]), verbose=0).flatten(), maximum_coefficients)
    return labels[np.argmin(mse(predicted_label, arr))]

def make_predict(image: np.array, arr: np.array, maximum_coefficients: np.array, q50_coeffs: np.array) -> np.array:
    w_mse_idx = make_predictWMSE(image, arr, maximum_coefficients, q50_coeffs)
    mse_idx = make_predictMSE(image, arr, maximum_coefficients)
    return w_mse_idx, mse_idx

def image_compression(image: np.array, *qfs: tuple) -> np.array:
    for qf in qfs:
        image = np.array(Image.open(io.BytesIO(cv2.imencode('.jpg', image, [int(cv2.IMWRITE_JPEG_QUALITY), qf])[1])))
    return image

In [43]:
maximum_coefficients, _ = max_min_coefficient(
    quality_range=(50, 100),
    n_coeffs=15
    )

In [44]:
model = tf.keras.models.load_model(
    model_file,
    custom_objects=({
        'custom_softmax': custom_softmax_activation(maximum_coefficients),
        'custom_two_terms_loss_wrapper': custom_two_terms_loss_wrapper(maximum_coefficients, 0.8),
        'custom_mse': custom_mse_wrapper(maximum_coefficients)
        })
    )

In [45]:
label2coefficient(
    model.predict(np.expand_dims(preprocess(image_file, (64, 64)), [0, -1])).flatten(), 
    max_coefficients=maximum_coefficients
    )

1/1 [==============================] - 1s 1s/step


array([ 7,  5,  4,  6,  4,  3,  7,  6,  4,  6,  8,  7,  7,  8, 11])

In [46]:
labels = _map[:, :2]
dct_coeffs = np.array([string2Q(i[2], size=(8, 8), flatten=True)[:15] for i in _map])
#Веса для WMSE
q50_coeffs = np.array([16,11,12,14,12,10,16,14,13,14,18,17,16,19,24], dtype='int64')

In [47]:
examples = [
    (65, ),
    (70, 90),
    (80, 90),
    (90, 90),
    (97, 90)
]

### A0: Программно сгенерировать 5 принципиально различных ситуаций: однократное сжатие, 𝑄𝐹1 < 𝑄𝐹2, 𝑄𝐹1 ≪ 𝑄𝐹2, 𝑄𝐹1 > 𝑄𝐹2, 𝑄𝐹1 ≈ 𝑄𝐹2. 

In [48]:
def a0_task(examples: list) -> pd.DataFrame:
    a0_data = []

    for example in examples:
        for path in glob("/content/gdrive/MyDrive/Colab Notebooks/images/*.tif"):
            image = image_compression(cv2.imread(path), *example)[:64, :64]
            pred_w, pred = make_predict(preprocess(image, (64, 64)), dct_coeffs, maximum_coefficients, q50_coeffs)
            a0_data.append([example[0], pred_w[0], np.abs(pred_w[0]-example[0]), pred[0], np.abs(pred[0]-example[0])])

    task_a0 = pd.DataFrame(a0_data, columns=['QF1', 'Prediction_WMSE', 'Error_WMSE', 'Prediction', 'Error'])
    return task_a0.groupby('QF1').mean().reset_index()

In [49]:
a0_task(examples)

,QF1,Prediction_WMSE,Error_WMSE,Prediction,Error
0,65,64.583333,1.250000,64.583333,1.250000
1,70,67.750000,3.083333,67.750000,3.083333
2,80,78.833333,1.833333,78.833333,1.833333
3,90,95.083333,5.083333,95.000000,5.000000
4,97,96.083333,0.916667,96.083333,0.916667


### A1: Сравнить ошибку при оценивании 𝑄𝐹1 по одному патчу и при оценивании по 𝑝 = 10 патчам, выбранным из одного изображения без пересечения.

In [50]:
def a1_task(examples: list) -> pd.DataFrame:
    a1_data = []

    for example in examples:
        for path in glob("/content/gdrive/MyDrive/Colab Notebooks/images/*.tif"):
            chunks = view_as_windows(image_compression(cv2.imread(path), *example), (64, 64, 3), 64).reshape(-1, 64, 64, 3)
            temp_datawmse = []
            temp_datamse = []
            
            for chunk in chunks[:10]:
                pred_w, pred = make_predict(preprocess(chunk, (64, 64)), dct_coeffs, maximum_coefficients, q50_coeffs)
                temp_datawmse.append([pred_w[0], np.abs(pred_w[0] - example[0])])
                temp_datamse.append([pred[0], np.abs(pred[0] - example[0])])
        
            mean_resultwmse = np.mean(temp_datawmse, axis=0)
            mean_resultmse = np.mean(temp_datamse, axis=0)
            a1_data.append([example[0], mean_resultwmse[0], mean_resultwmse[1], mean_resultmse[0], mean_resultmse[1]])
    task_a1 = pd.DataFrame(a1_data, columns=['QF1', 'Prediction_WMSE', 'Error_WMSE', 'Prediction', 'Error'])
    return task_a1.groupby('QF1').mean().reset_index()

In [51]:
a1_task(examples)

,QF1,Prediction_WMSE,Error_WMSE,Prediction,Error
0,65,64.366667,1.266667,64.350000,1.283333
1,70,68.075000,2.541667,68.083333,2.533333
2,80,78.125000,2.325000,78.083333,2.333333
3,90,94.441667,4.825000,94.408333,4.791667
4,97,95.366667,1.633333,95.358333,1.641667
